In [2]:
import numpy as np
import pandas as pd

path = '/Users/jsha/gjai/final_project/black_swan_physionet/'

file1 = 'train_preprocessing_done.csv'
file2 = 'valid_preprocessing_done.csv'
file3 = 'test_preprocessing_done.csv'

train_df = pd.read_csv(path+file1)
valid_df = pd.read_csv(path+file2)
test_df = pd.read_csv(path+file3)

In [4]:
column_list = train_df.columns.to_list()
column_list

['pid',
 'age',
 'Gender',
 'ALP',
 'ALT',
 'AST',
 'Albumin',
 'BUN',
 'Bilirubin',
 'Cholesterol',
 'Creatinine',
 'FiO2',
 'GCS',
 'Glucose',
 'HCO3',
 'HCT',
 'HR',
 'K',
 'Lactate',
 'Mg',
 'Na',
 'PaCO2',
 'PaO2',
 'Platelets',
 'RR',
 'SPO2',
 'BT',
 'TroponinI',
 'TroponinT',
 'Urine',
 'WBC',
 'pH',
 'MechVent',
 'survival',
 'SBP',
 'DBP',
 'MBP',
 'male',
 'female',
 'surv_los',
 'LOS',
 'labels',
 'ICUType_1.0',
 'ICUType_2.0',
 'ICUType_3.0',
 'ICUType_4.0']

In [13]:
# 1. **Respiration:** PaO2 / FiO2, Mechanical Ventilation(yes or no)
# 2. **Coagulation:** Platelets
# 3. **Liver:** Bilirubin
# 4. **Neurological:** Glasgow coma score(GCS)
# 5. **Cardiovascular:** MAP, Vasopressors(yes or no) -> MAP은 안보여서 MBP쓰자. SBP(침습성수축기동맥혈압), DBP(침습성확장기동맥혈압), MBP(침습성평균동맥혈압???)
# 6. **Renal:** Creatinine, Urine output
dummy = 'pid,PaO2,FiO2,MechVent,Platelets,Bilirubin,GCS,MBP,Creatinine,Urine'
col_list = dummy.split(',')
col_list

['pid',
 'PaO2',
 'FiO2',
 'MechVent',
 'Platelets',
 'Bilirubin',
 'GCS',
 'MBP',
 'Creatinine',
 'Urine']

In [14]:
train_df2 = train_df[col_list]
train_df2.isnull().sum()

pid           0
PaO2          0
FiO2          0
MechVent      0
Platelets     0
Bilirubin     0
GCS           0
MBP           0
Creatinine    0
Urine         0
dtype: int64

In [17]:
train_df3 = train_df2.groupby(by='pid').max()
train_df3

,PaO2,FiO2,MechVent,Platelets,Bilirubin,GCS,MBP,Creatinine,Urine
pid,,,,,,,,,
132540.0,445.0,1.0,1.0,226.0,0.9,15.0,100.0,1.2,770.0
132543.0,122.0,0.5,1.0,391.0,0.9,15.0,77.0,1.0,700.0
132545.0,122.0,0.5,1.0,173.0,0.9,15.0,77.0,1.0,220.0
132547.0,179.0,0.5,1.0,276.0,0.9,13.0,107.0,1.4,1200.0
132548.0,122.0,0.5,1.0,337.0,0.9,15.0,174.0,4.1,190.0
...,...,...,...,...,...,...,...,...,...
142664.0,304.0,0.5,1.0,205.0,0.9,13.0,108.0,1.0,640.0
142665.0,205.0,0.5,1.0,218.0,0.9,15.0,99.0,1.0,150.0
142667.0,122.0,0.5,1.0,173.0,0.9,15.0,77.0,1.0,900.0


In [23]:
def sofa_score(df):
    sfs = pd.DataFrame(index=range(len(df)), columns=['sofa_score'])
    

train_df3.iloc[0,:]

PaO2          445.0
FiO2            1.0
MechVent        1.0
Platelets     226.0
Bilirubin       0.9
GCS            15.0
MBP           100.0
Creatinine      1.2
Urine         770.0
Name: 132540.0, dtype: float64

In [65]:
train_df3.MechVent.value_counts()

1.0    3240
Name: MechVent, dtype: int64

In [67]:
cal_respi = train_df3.PaO2 / train_df3.FiO2
# Respiratory
respi = cal_respi.copy()
respi[:] = 0
respi[cal_respi < 400] = 1
respi[cal_respi < 300] = 2
respi[(cal_respi < 200) & (train_df.MechVent == 1)] = 3
respi[(cal_respi < 100) & (train_df.MechVent == 1)] = 4
# respi.value_counts()



# print(sfs)
# cal_respiratory = train_df3.PaO2 / train_df3.FiO2
# sfs[cal_respiratory >= 400] = 0
# sfs[cal_respiratory >= 400] 
# cal_respiratory[cal_respiratory >= 400] = None
# sfs.respiratory[cal_respiratory < 400] = 1
# cal_respiratory[cal_respiratory < 400] = None
# sfs.respiratory[cal_respiratory < 300] = 2
# cal_respiratory[cal_respiratory < 300] = None
# sfs.respiratory[cal_respiratory < 200] = 3
# cal_respiratory[cal_respiratory < 200] = None
# sfs.respiratory[cal_respiratory < 100] = 4
# cal_respiratory[cal_respiratory < 100] = None




# coagulation = Platelets
# liver = Bilirubin
# cardiovascular = MBP
# gcs = GCS
# renal = Creatinine
# # print(respiratory)
# # print(respiratory.value_counts())

# respiratory[respiratory == 0]



2.0    1611
0.0     663
1.0     564
3.0     402
dtype: int64

In [26]:
sfs

,sofa_score
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
3235,NaN
3236,NaN
3237,NaN
3238,NaN
